<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Hate_BR_BERT_com_pre_processamento_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn
!pip install -U spacy
!python -m spacy download pt


!pip install wandb
!pip install --upgrade simpletransformers
!pip install --upgrade transformers

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression



from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle
import spacy
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
from sklearn.model_selection import train_test_split


In [2]:
base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')


In [3]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

# print(preprocess(base_hatebr['instagram_comments']))
base_hatebr['preprocessed_comments'] =  base_hatebr['instagram_comments'].apply(preprocess)

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
base_hatebr_train_data = base_hatebr.iloc[:,[4, 1]]
train,eva = train_test_split(base_hatebr_train_data,test_size = 0.2)

train_df = pd.DataFrame({
    'text': train['preprocessed_comments'],
    'label': train['offensive_language']
})

eval_df = pd.DataFrame({
    'text': eva['preprocessed_comments'],
    'label': eva['offensive_language']
})

# #Optional model configuration
model_args = ClassificationArgs(
      num_train_epochs=1
      # overwrite_output_dir=True
  )

# #Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)
model.train_model(train_df)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5600 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/700 [00:00<?, ?it/s]

(700, 0.35175816857655134)

In [5]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.accuracy_score(true,predicted)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1400 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/175 [00:00<?, ?it/s]

0.9021428571428571

Score 0.90

In [6]:
with open('Hate_BR-BERT-preprocess', 'wb') as file:
    pickle.dump(model, file)